# Week 4 Exercise - Code Generator Toolbox
### Author: Samuel Kalu, Team Euclid, Week 4

This notebook implements a comprehensive code generation toolbox with:
- **Python to C++ Converter**: Generate high-performance C++ code from Python
- **Python to Go Converter**: Generate efficient Go code from Python
- **Docstring & Comment Generator**: Add PEP-257 compliant docstrings and inline comments
- **Unit Test Generator**: Generate comprehensive pytest unit tests

Features:
- Gradio UI with multiple tabs
- Streaming responses
- Model switching (OpenAI GPT, Ollama, Google Gemini)
- Error handling and validation

## 1. Setup and Configuration

In [ ]:
# imports
import os
import json
import subprocess
import tempfile
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import Markdown, display

# Load environment variables
load_dotenv(override=True)

# API Keys
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
ollama_api_key = os.getenv('OLLAMA_API_KEY', 'ollama')

# Verify API keys
if openai_api_key:
    print(f"✓ OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("✗ OpenAI API Key not set")

if google_api_key:
    print(f"✓ Google API Key exists and begins {google_api_key[:8]}")
else:
    print("ℹ Google API Key not set (optional)")

if ollama_api_key:
    print(f"✓ Ollama API Key exists")
else:
    print("ℹ Ollama API Key not set (optional - requires local Ollama installation)")

In [ ]:
# Initialize API clients

# OpenAI client
openai_client = OpenAI(api_key=openai_api_key) if openai_api_key else None

# Google Gemini client (via OpenAI-compatible API)
gemini_client = None
if google_api_key:
    gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
    gemini_client = OpenAI(api_key=google_api_key, base_url=gemini_url)

# Ollama client (local)
ollama_client = None
ollama_url = "http://localhost:11434/v1"
try:
    ollama_client = OpenAI(api_key=ollama_api_key, base_url=ollama_url)
    print("✓ Ollama client initialized")
except Exception as e:
    print(f"ℹ Ollama not available: {e}")

# Available models configuration
MODELS = {
    "OpenAI": ["gpt-4.1-mini", "gpt-4o", "gpt-4.1-nano"],
    "Gemini": ["gemini-2.5-pro", "gemini-2.0-flash"],
    "Ollama": ["llama3.1", "llama3.2", "codellama"]
}

ALL_MODELS = []
if openai_client:
    ALL_MODELS.extend(MODELS["OpenAI"])
if gemini_client:
    ALL_MODELS.extend(MODELS["Gemini"])
if ollama_client:
    ALL_MODELS.extend(MODELS["Ollama"])

DEFAULT_MODEL = ALL_MODELS[0] if ALL_MODELS else "llama3.1"

print(f"\nAvailable models: {ALL_MODELS}")
print(f"Default model: {DEFAULT_MODEL}")

In [ ]:
# Helper functions to get the right client for a model

def get_client_for_model(model_name: str):
    """Returns the appropriate API client for the given model name."""
    if model_name in MODELS["OpenAI"]:
        return openai_client
    elif model_name in MODELS["Gemini"]:
        return gemini_client
    elif model_name in MODELS["Ollama"]:
        return ollama_client
    return openai_client  # fallback

def clean_code_output(code: str) -> str:
    """Remove markdown code blocks from the output."""
    if "```" in code:
        # Remove ```language and ``` markers
        lines = code.split('\n')
        cleaned = []
        in_block = False
        for line in lines:
            if line.strip().startswith('```'):
                in_block = not in_block
                continue
            cleaned.append(line)
        return '\n'.join(cleaned)
    return code

# Sample Python code for testing
SAMPLE_PYTHON_CODE = '''
def fibonacci(n):
    """Calculate the nth Fibonacci number."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b

def calculate_sum(numbers):
    """Calculate the sum of a list of numbers."""
    total = 0
    for num in numbers:
        total += num
    return total

if __name__ == "__main__":
    print(f"Fibonacci(10) = {fibonacci(10)}")
    print(f"Sum of [1, 2, 3, 4, 5] = {calculate_sum([1, 2, 3, 4, 5])}")
'''

print("Sample Python code loaded for testing.")

## 2. Python to C++ Converter

In [ ]:
# System prompt for Python to C++ conversion

CPP_SYSTEM_PROMPT = """You are an expert C++ developer specializing in high-performance code optimization.
Your task is to convert Python code into efficient, modern C++ code (C++17 or later).

Guidelines:
1. Preserve the exact functionality and logic of the original Python code
2. Use modern C++ features (auto, smart pointers, ranges, etc.)
3. Optimize for performance while maintaining readability
4. Include necessary headers and a main() function for testing
5. Use appropriate C++ data types and STL containers
6. Add brief comments only where the logic is non-obvious
7. Do NOT include explanations outside the code

Respond ONLY with the C++ code, no additional text."""

def convert_to_cpp_streaming(python_code: str, model_name: str):
    """Convert Python code to C++ with streaming output."""
    client = get_client_for_model(model_name)
    
    if not client:
        yield "Error: No API client available. Please check your API keys."
        return
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": CPP_SYSTEM_PROMPT},
                {"role": "user", "content": f"Convert this Python code to C++:\n\n{python_code}"}
            ],
            stream=True,
            temperature=0.3
        )
        
        result = ""
        for chunk in response:
            if chunk.choices[0].delta.content:
                result += chunk.choices[0].delta.content
                yield clean_code_output(result)
                
    except Exception as e:
        yield f"Error during conversion: {str(e)}"

def convert_to_cpp_batch(python_code: str, model_name: str) -> str:
    """Convert Python code to C++ without streaming."""
    client = get_client_for_model(model_name)
    
    if not client:
        return "Error: No API client available."
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": CPP_SYSTEM_PROMPT},
                {"role": "user", "content": f"Convert this Python code to C++:\n\n{python_code}"}
            ],
            temperature=0.3
        )
        
        return clean_code_output(response.choices[0].message.content)
        
    except Exception as e:
        return f"Error during conversion: {str(e)}"

print("✓ Python to C++ converter functions loaded")

## 3. Python to Go Converter

In [ ]:
# System prompt for Python to Go conversion

GO_SYSTEM_PROMPT = """You are an expert Go developer specializing in clean, idiomatic, and performant code.
Your task is to convert Python code into efficient, idiomatic Go code.

Guidelines:
1. Preserve the exact functionality and logic of the original Python code
2. Use idiomatic Go patterns and conventions
3. Include proper error handling (return errors, don't panic)
4. Include necessary imports and a main() function for testing
5. Use appropriate Go data types and standard library packages
6. Add brief comments only where the logic is non-obvious
7. Do NOT include explanations outside the code

Respond ONLY with the Go code, no additional text."""

def convert_to_go_streaming(python_code: str, model_name: str):
    """Convert Python code to Go with streaming output."""
    client = get_client_for_model(model_name)
    
    if not client:
        yield "Error: No API client available. Please check your API keys."
        return
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": GO_SYSTEM_PROMPT},
                {"role": "user", "content": f"Convert this Python code to Go:\n\n{python_code}"}
            ],
            stream=True,
            temperature=0.3
        )
        
        result = ""
        for chunk in response:
            if chunk.choices[0].delta.content:
                result += chunk.choices[0].delta.content
                yield clean_code_output(result)
                
    except Exception as e:
        yield f"Error during conversion: {str(e)}"

def convert_to_go_batch(python_code: str, model_name: str) -> str:
    """Convert Python code to Go without streaming."""
    client = get_client_for_model(model_name)
    
    if not client:
        return "Error: No API client available."
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": GO_SYSTEM_PROMPT},
                {"role": "user", "content": f"Convert this Python code to Go:\n\n{python_code}"}
            ],
            temperature=0.3
        )
        
        return clean_code_output(response.choices[0].message.content)
        
    except Exception as e:
        return f"Error during conversion: {str(e)}"

print("✓ Python to Go converter functions loaded")

## 4. Docstring & Comment Generator

In [ ]:
# System prompt for docstring generation

DOCSTRING_SYSTEM_PROMPT = """You are an expert Python developer and code reviewer.
Your job is to read the user's provided Python function and return:
1. A concise, PEP-257-compliant docstring summarizing what the function does
2. Clarify parameter types, return values, and side effects
3. Helpful inline comments that improve readability and maintainability

Guidelines:
- Do NOT modify variable names or refactor the function logic
- Only output the improved function with docstring and comments
- Don't be extremely verbose - keep comments meaningful but concise
- Write at a senior developer level

Respond ONLY with the improved Python code, no additional text."""

def generate_docstring_streaming(code: str, model_name: str):
    """Generate docstrings and comments with streaming output."""
    client = get_client_for_model(model_name)
    
    if not client:
        yield "Error: No API client available. Please check your API keys."
        return
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": DOCSTRING_SYSTEM_PROMPT},
                {"role": "user", "content": f"Improve this Python code with docstrings and comments:\n\n{code}"}
            ],
            stream=True,
            temperature=0.2
        )
        
        result = ""
        for chunk in response:
            if chunk.choices[0].delta.content:
                result += chunk.choices[0].delta.content
                yield clean_code_output(result)
                
    except Exception as e:
        yield f"Error during generation: {str(e)}"

def generate_docstring_batch(code: str, model_name: str) -> str:
    """Generate docstrings and comments without streaming."""
    client = get_client_for_model(model_name)
    
    if not client:
        return "Error: No API client available."
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": DOCSTRING_SYSTEM_PROMPT},
                {"role": "user", "content": f"Improve this Python code with docstrings and comments:\n\n{code}"}
            ],
            temperature=0.2
        )
        
        return clean_code_output(response.choices[0].message.content)
        
    except Exception as e:
        return f"Error during generation: {str(e)}"

print("✓ Docstring generator functions loaded")

## 5. Unit Test Generator

In [ ]:
# System prompt for unit test generation

UNITTEST_SYSTEM_PROMPT = """You are a seasoned Python developer and testing expert.
Your task is to read the user's provided Python function and generate comprehensive unit tests.

Guidelines:
1. Write tests using pytest framework (or unittest if pytest is not appropriate)
2. Include typical cases, edge cases, and error cases
3. Use clear, descriptive test function names
4. Include minimal necessary setup code (avoid over-mocking)
5. If dependencies or mocking are needed, include proper imports
6. Do NOT change the original function - focus only on test coverage

Respond ONLY with the test code, no additional text."""

def generate_unittest_streaming(code: str, model_name: str):
    """Generate unit tests with streaming output."""
    client = get_client_for_model(model_name)
    
    if not client:
        yield "Error: No API client available. Please check your API keys."
        return
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": UNITTEST_SYSTEM_PROMPT},
                {"role": "user", "content": f"Generate comprehensive unit tests for this Python function:\n\n{code}"}
            ],
            stream=True,
            temperature=0.2
        )
        
        result = ""
        for chunk in response:
            if chunk.choices[0].delta.content:
                result += chunk.choices[0].delta.content
                yield clean_code_output(result)
                
    except Exception as e:
        yield f"Error during generation: {str(e)}"

def generate_unittest_batch(code: str, model_name: str) -> str:
    """Generate unit tests without streaming."""
    client = get_client_for_model(model_name)
    
    if not client:
        return "Error: No API client available."
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": UNITTEST_SYSTEM_PROMPT},
                {"role": "user", "content": f"Generate comprehensive unit tests for this Python function:\n\n{code}"}
            ],
            temperature=0.2
        )
        
        return clean_code_output(response.choices[0].message.content)
        
    except Exception as e:
        return f"Error during generation: {str(e)}"

print("✓ Unit test generator functions loaded")

## 6. Gradio UI Implementation

In [ ]:
# Gradio UI with multiple tabs

def create_gradio_ui():
    """Create and configure the Gradio interface."""
    
    with gr.Blocks(
        theme=gr.themes.Soft(
            primary_hue=gr.themes.colors.blue,
            spacing_size=gr.themes.sizes.spacing_md,
            radius_size=gr.themes.sizes.radius_md
        ),
        title="Code Generator Toolbox - Samuel Kalu, Team Euclid"
    ) as ui:
        
        gr.Markdown(
            """
            # Code Generator Toolbox
            ### By Samuel Kalu, Team Euclid - Week 4
            
            Convert Python code to C++ or Go, generate docstrings, and create unit tests.
            Supports multiple models: OpenAI GPT, Google Gemini, and Ollama (local).
            """
        )
        
        # Tab 1: Python to C++ Converter
        with gr.Tab("Python to C++"):
            gr.Markdown("### Convert Python code to high-performance C++ (C++17+)")
            
            with gr.Row():
                with gr.Column(scale=1):
                    cpp_input = gr.Code(
                        label="Python Code",
                        language="python",
                        lines=15,
                        value=SAMPLE_PYTHON_CODE
                    )
                    cpp_model = gr.Dropdown(
                        choices=ALL_MODELS,
                        value=DEFAULT_MODEL,
                        label="Select Model"
                    )
                    cpp_btn = gr.Button("Convert to C++", variant="primary")
                with gr.Column(scale=1):
                    cpp_output = gr.Code(
                        label="C++ Output",
                        language="cpp",
                        lines=20
                    )
            
            cpp_btn.click(
                fn=convert_to_cpp_batch,
                inputs=[cpp_input, cpp_model],
                outputs=cpp_output
            )
        
        # Tab 2: Python to Go Converter
        with gr.Tab("Python to Go"):
            gr.Markdown("### Convert Python code to idiomatic Go")
            
            with gr.Row():
                with gr.Column(scale=1):
                    go_input = gr.Code(
                        label="Python Code",
                        language="python",
                        lines=15,
                        value=SAMPLE_PYTHON_CODE
                    )
                    go_model = gr.Dropdown(
                        choices=ALL_MODELS,
                        value=DEFAULT_MODEL,
                        label="Select Model"
                    )
                    go_btn = gr.Button("Convert to Go", variant="primary")
                with gr.Column(scale=1):
                    go_output = gr.Code(
                        label="Go Output",
                        language="go",
                        lines=20
                    )
            
            go_btn.click(
                fn=convert_to_go_batch,
                inputs=[go_input, go_model],
                outputs=go_output
            )
        
        # Tab 3: Docstring Generator
        with gr.Tab("Docstring Generator"):
            gr.Markdown("### Add PEP-257 compliant docstrings and inline comments")
            
            with gr.Row():
                with gr.Column(scale=1):
                    doc_input = gr.Code(
                        label="Python Code",
                        language="python",
                        lines=15,
                        value=SAMPLE_PYTHON_CODE
                    )
                    doc_model = gr.Dropdown(
                        choices=ALL_MODELS,
                        value=DEFAULT_MODEL,
                        label="Select Model"
                    )
                    doc_btn = gr.Button("Generate Docstrings", variant="primary")
                with gr.Column(scale=1):
                    doc_output = gr.Code(
                        label="Improved Code with Docstrings",
                        language="python",
                        lines=20
                    )
            
            doc_btn.click(
                fn=generate_docstring_batch,
                inputs=[doc_input, doc_model],
                outputs=doc_output
            )
        
        # Tab 4: Unit Test Generator
        with gr.Tab("🧪 Unit Test Generator"):
            gr.Markdown("### Generate comprehensive pytest unit tests")
            
            with gr.Row():
                with gr.Column(scale=1):
                    test_input = gr.Code(
                        label="Python Code to Test",
                        language="python",
                        lines=15,
                        value=SAMPLE_PYTHON_CODE
                    )
                    test_model = gr.Dropdown(
                        choices=ALL_MODELS,
                        value=DEFAULT_MODEL,
                        label="Select Model"
                    )
                    test_btn = gr.Button("🧪 Generate Tests", variant="primary")
                with gr.Column(scale=1):
                    test_output = gr.Code(
                        label="Generated Unit Tests",
                        language="python",
                        lines=20
                    )
            
            test_btn.click(
                fn=generate_unittest_batch,
                inputs=[test_input, test_model],
                outputs=test_output
            )
        
        # Footer
        gr.Markdown(
            """
            ---
            **Tips:**
            - For best results, use GPT-4 or Gemini 2.5 Pro models
            - Ollama models run locally (requires Ollama installation)
            - Streaming is available in the backend functions for real-time generation
            """
        )
    
    return ui

print("✓ Gradio UI configuration complete")

## 7. Demo and Testing

In [ ]:
# Quick demo without launching UI

if openai_client:
    print("\n=== Testing Python to C++ Conversion ===")
    cpp_result = convert_to_cpp_batch(SAMPLE_PYTHON_CODE, "gpt-4.1-mini")
    print(cpp_result[:500] + "..." if len(cpp_result) > 500 else cpp_result)
    
    print("\n=== Testing Docstring Generation ===")
    doc_result = generate_docstring_batch(SAMPLE_PYTHON_CODE, "gpt-4.1-mini")
    print(doc_result[:500] + "..." if len(doc_result) > 500 else doc_result)
else:
    print("OpenAI client not available. Skipping demo.")

## 8. Launch the Application

In [ ]:
# Launch the Gradio UI

if __name__ == "__main__":
    ui = create_gradio_ui()
    print("\n🚀 Launching Code Generator Toolbox...")
    print("Open the URL shown below in your browser to use the application.")
    print("Press Ctrl+C to stop the server.\n")
    ui.launch(share=False, inbrowser=True)

## Summary

This Week 4 exercise solution demonstrates:

1. **Frontier Model Integration**: Uses OpenAI GPT, Google Gemini, and Ollama models
2. **Code Generation**: Converts Python to C++ and Go with optimized output
3. **Code Documentation**: Generates PEP-257 compliant docstrings and inline comments
4. **Test Generation**: Creates comprehensive pytest unit tests
5. **Gradio UI**: Clean, tabbed interface with model switching
6. **Streaming Support**: Backend functions support both streaming and batch modes
7. **Error Handling**: Graceful handling of API failures and missing keys

### Key Features:
- ✅ Multiple model support (OpenAI, Gemini, Ollama)
- ✅ Streaming responses for real-time generation
- ✅ Clean code output (removes markdown artifacts)
- ✅ Modular architecture (easy to add new converters)
- ✅ Professional UI with Gradio
- ✅ Comprehensive error handling

### Author: Samuel Kalu, Team Euclid, Week 4